In [ ]:
# | default_exp blocks/transformer

# Imports

In [ ]:
# | export


from typing import Literal

import torch
from einops import rearrange
from torch import nn

from vision_architectures.layers.attention import Attention1D, Attention1DConfig, Attention3D, Attention3DConfig
from vision_architectures.layers.embeddings import RelativePositionEmbeddings
from vision_architectures.utils.activation_checkpointing import ActivationCheckpointing
from vision_architectures.utils.activations import get_act_layer
from vision_architectures.utils.custom_base_model import CustomBaseModel
from vision_architectures.utils.rearrange import make_channels_first, make_channels_last
from vision_architectures.utils.residuals import Residual

# Configs

In [ ]:
# | export


class Attention1DMLPConfig(CustomBaseModel):
    dim: int
    mlp_ratio: int = 4
    activation: str = "gelu"
    mlp_drop_prob: float = 0.0


class Attention3DMLPConfig(Attention1DMLPConfig):
    pass


class Attention1DWithMLPConfig(Attention1DConfig, Attention1DMLPConfig):
    dim: int | tuple[int, int]
    norm_location: Literal["pre", "post"] = "post"
    layer_norm_eps: float = 1e-6


class Attention3DWithMLPConfig(Attention3DConfig, Attention3DMLPConfig):
    dim: int | tuple[int, int]
    norm_location: Literal["pre", "post"] = "post"
    layer_norm_eps: float = 1e-6

# Architecture

In [ ]:
# | export


class Attention1DMLP(nn.Module):
    def __init__(self, config: Attention1DMLPConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = Attention1DMLPConfig.model_validate(config | kwargs)

        dim = self.config.dim
        mlp_ratio = self.config.mlp_ratio
        activation = self.config.activation
        mlp_drop_prob = self.config.mlp_drop_prob

        self.dense1 = nn.Linear(dim, dim * mlp_ratio)

        if isinstance(activation, nn.Module):
            self.act = activation
        else:
            self.act = get_act_layer(activation)

        self.dense2 = nn.Linear(dim * mlp_ratio, dim)
        self.dropout = nn.Dropout(mlp_drop_prob)

        self.checkpointing_level1 = ActivationCheckpointing(1, checkpointing_level)
        self.checkpointing_level2 = ActivationCheckpointing(2, checkpointing_level)

    def _forward(self, hidden_states: torch.Tensor):
        # hidden_states: (b, T, dim)
        def first_half(hidden_states):
            hidden_states = self.dense1(hidden_states)
            hidden_states = self.act(hidden_states)
            return hidden_states

        def second_half(hidden_states):
            hidden_states = self.dense2(hidden_states)
            hidden_states = self.dropout(hidden_states)
            return hidden_states

        hidden_states = self.checkpointing_level1(first_half, hidden_states)
        hidden_states = self.checkpointing_level1(second_half, hidden_states)
        return hidden_states

    def forward(self, *args, **kwargs):
        return self.checkpointing_level2(self._forward, *args, **kwargs)

In [ ]:
test = Attention1DMLP(dim=64, activation="relu", mlp_drop_prob=0.2)
display(test)
display(test(torch.randn(2, 28, 64)).shape)


Attention1DMLP(
  (dense1): Linear(in_features=64, out_features=256, bias=True)
  (act): ReLU()
  (dense2): Linear(in_features=256, out_features=64, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (checkpointing_level1): ActivationCheckpointing(enabled=False, checkpointing_level=1)
  (checkpointing_level2): ActivationCheckpointing(enabled=False, checkpointing_level=2)
)

torch.Size([2, 28, 64])

In [ ]:
# | export


class Attention3DMLP(Attention1DMLP):
    def __init__(self, config: Attention3DMLPConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__(config, checkpointing_level, **kwargs)

    def _forward(self, hidden_states: torch.Tensor, channels_first: bool = True):
        # hidden_states: (b, dim, z, y, x) or (b, z, y, x, dim)

        if channels_first:
            hidden_states = make_channels_last(hidden_states)

        hidden_states = super()._forward(hidden_states)

        if channels_first:
            hidden_states = make_channels_first(hidden_states)

        return hidden_states

    def forward(self, *args, **kwargs):
        return self.checkpointing_level1(self._forward, *args, **kwargs)

In [ ]:
test = Attention3DMLP(dim=64)
display(test)
display(test(torch.randn(2, 64, 4, 4, 4)).shape)


Attention3DMLP(
  (dense1): Linear(in_features=64, out_features=256, bias=True)
  (act): GELU(approximate='none')
  (dense2): Linear(in_features=256, out_features=64, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (checkpointing_level1): ActivationCheckpointing(enabled=False, checkpointing_level=1)
  (checkpointing_level2): ActivationCheckpointing(enabled=False, checkpointing_level=2)
)

torch.Size([2, 64, 4, 4, 4])

In [ ]:
# | export


class Attention1DWithMLP(nn.Module):
    def __init__(
        self,
        config: Attention1DWithMLPConfig = {},
        relative_position_bias: RelativePositionEmbeddings | None = None,
        logit_scale: float | None = None,
        checkpointing_level: int = 0,
        **kwargs
    ):
        super().__init__()

        self.config = Attention1DWithMLPConfig.model_validate(config | kwargs)

        dim_qk = self.config.dim_qk
        layer_norm_eps = self.config.layer_norm_eps

        self.attn = Attention1D(
            self.config,
            relative_position_bias=relative_position_bias,
            logit_scale=logit_scale,
            checkpointing_level=checkpointing_level,
        )
        self.layernorm1 = nn.LayerNorm(dim_qk, eps=layer_norm_eps)
        self.mlp = Attention1DMLP(self.config, checkpointing_level=checkpointing_level)
        self.layernorm2 = nn.LayerNorm(dim_qk, eps=layer_norm_eps)

        self.residual = Residual()

        self.checkpointing_level3 = ActivationCheckpointing(3, checkpointing_level)

    def _forward(self, query: torch.Tensor, key: torch.Tensor, value: torch.Tensor):
        # Each is (b, T, dim)
        res_connection1 = query
        # (b, T, dim)

        if self.config.norm_location == "pre":
            query = self.layernorm1(query)
            key = self.layernorm1(key)
            value = self.layernorm1(value)
            # (b, T, dim)

        hidden_states = self.attn(query, key, value)
        # (b, T, dim)

        if self.config.norm_location == "post":
            hidden_states = self.layernorm1(hidden_states)
            # (b, T, dim)

        hidden_states = self.residual(res_connection1, hidden_states)
        res_connection2 = hidden_states
        # (b, T, dim)

        if self.config.norm_location == "pre":
            hidden_states = self.layernorm2(hidden_states)
            # (b, T, dim)

        hidden_states = self.mlp(hidden_states)
        # (b, T, dim)

        if self.config.norm_location == "post":
            hidden_states = self.layernorm2(hidden_states)
            # (b, T, dim)

        hidden_states = self.residual(res_connection2, hidden_states)
        # (b, T, dim)

        return hidden_states

    def forward(self, *args, **kwargs):
        return self.checkpointing_level3(self._forward, *args, **kwargs)

In [ ]:
test = Attention1DWithMLP(dim=54, num_heads=3)
output = torch.randn(2, 64, 54)

display(test)
display(test(output, output, output).shape)


Attention1DWithMLP(
  (attn): Attention1D(
    (W_q): Linear(in_features=54, out_features=54, bias=True)
    (W_k): Linear(in_features=54, out_features=54, bias=True)
    (W_v): Linear(in_features=54, out_features=54, bias=True)
    (proj): Linear(in_features=54, out_features=54, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm1): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
  (mlp): Attention1DMLP(
    (dense1): Linear(in_features=54, out_features=216, bias=True)
    (act): GELU(approximate='none')
    (dense2): Linear(in_features=216, out_features=54, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (layernorm2): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
  (residual): Residual()
)

torch.Size([2, 64, 54])

In [ ]:
# | export


class Attention3DWithMLP(nn.Module):
    def __init__(
        self,
        config: Attention3DWithMLPConfig = {},
        relative_position_bias: RelativePositionEmbeddings | None = None,
        logit_scale: float | None = None,
        checkpointing_level: int = 0,
        **kwargs
    ):
        super().__init__()

        self.config = Attention3DWithMLPConfig.model_validate(config | kwargs)

        dim_qk = self.config.dim_qk
        layer_norm_eps = self.config.layer_norm_eps

        self.attn = Attention3D(
            self.config,
            relative_position_bias=relative_position_bias,
            logit_scale=logit_scale,
            checkpointing_level=checkpointing_level,
        )
        self.layernorm1 = nn.LayerNorm(dim_qk, eps=layer_norm_eps)
        self.mlp = Attention3DMLP(self.config, checkpointing_level=checkpointing_level)
        self.layernorm2 = nn.LayerNorm(dim_qk, eps=layer_norm_eps)

        self.residual = Residual()

        self.checkpointing_level3 = ActivationCheckpointing(3, checkpointing_level)

    def _forward(
        self,
        query: torch.Tensor,
        key: torch.Tensor,
        value: torch.Tensor,
        channels_first: bool = True,
    ):
        # Each is (b, [dim], tokens_z, tokens_y, tokens_x, [dim])

        if channels_first:
            query = rearrange(query, "b d z y x -> b z y x d").contiguous()
            key = rearrange(key, "b d z y x -> b z y x d").contiguous()
            value = rearrange(value, "b d z y x -> b z y x d").contiguous()
            # (b, tokens_z, tokens_y, tokens_x, dim)

        res_connection1 = query
        # (b, tokens_z, tokens_y, tokens_x, dim)

        if self.config.norm_location == "pre":
            query = self.layernorm1(query)
            key = self.layernorm1(key)
            value = self.layernorm1(value)
            # (b, tokens_z, tokens_y, tokens_x, dim)

        hidden_states = self.attn(query, key, value, channels_first=False)
        # (b, tokens_z, tokens_y, tokens_x, dim)

        if self.config.norm_location == "post":
            hidden_states = self.layernorm1(hidden_states)
            # (b, tokens_z, tokens_y, tokens_x, dim)

        hidden_states = self.residual(res_connection1, hidden_states)
        res_connection2 = hidden_states
        # (b, tokens_z, tokens_y, tokens_x, dim)

        if self.config.norm_location == "pre":
            hidden_states = self.layernorm2(hidden_states)
            # (b, tokens_z, tokens_y, tokens_x, dim)

        hidden_states = self.mlp(hidden_states, channels_first=False)
        # (b, tokens_z, tokens_y, tokens_x, dim)

        if self.config.norm_location == "post":
            hidden_states = self.layernorm2(hidden_states)
            # (b, tokens_z, tokens_y, tokens_x, dim)

        hidden_states = self.residual(res_connection2, hidden_states)
        # (b, tokens_z, tokens_y, tokens_x, dim)

        if channels_first:
            hidden_states = rearrange(hidden_states, "b z y x d -> b d z y x").contiguous()
            # (b, dim, tokens_z, tokens_y, tokens_x)

        return hidden_states

    def forward(self, *args, **kwargs):
        return self.checkpointing_level3(self._forward, *args, **kwargs)

In [ ]:
test = Attention3DWithMLP(dim=54, num_heads=3)
output = torch.randn(2, 54, 4, 4, 4)

display(test)
display(test(output, output, output, channels_first=True).shape)


Attention3DWithMLP(
  (attn): Attention3D(
    (W_q): Linear(in_features=54, out_features=54, bias=True)
    (W_k): Linear(in_features=54, out_features=54, bias=True)
    (W_v): Linear(in_features=54, out_features=54, bias=True)
    (proj): Linear(in_features=54, out_features=54, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm1): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
  (mlp): Attention3DMLP(
    (dense1): Linear(in_features=54, out_features=216, bias=True)
    (act): GELU(approximate='none')
    (dense2): Linear(in_features=216, out_features=54, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (layernorm2): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
  (residual): Residual()
)

torch.Size([2, 54, 4, 4, 4])

In [ ]:
# | export


class TransformerEncoderBlock1D(Attention1DWithMLP):
    def forward(self, qkv: torch.Tensor):
        # qkv: (b, num_tokens, dim)
        return super().forward(qkv, qkv, qkv)

In [ ]:
test = TransformerEncoderBlock1D(dim=54, num_heads=6, mlp_ratio=2)
display(test)
o = test(torch.randn(2, 64, 54))
display(o.shape)


TransformerEncoderBlock1D(
  (attn): Attention1D(
    (W_q): Linear(in_features=54, out_features=54, bias=True)
    (W_k): Linear(in_features=54, out_features=54, bias=True)
    (W_v): Linear(in_features=54, out_features=54, bias=True)
    (proj): Linear(in_features=54, out_features=54, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm1): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
  (mlp): Attention1DMLP(
    (dense1): Linear(in_features=54, out_features=108, bias=True)
    (act): GELU(approximate='none')
    (dense2): Linear(in_features=108, out_features=54, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (layernorm2): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
  (residual): Residual()
)

torch.Size([2, 64, 54])

In [ ]:
# | export


class TransformerEncoderBlock3D(Attention3DWithMLP):
    def forward(self, qkv: torch.Tensor, channels_first: bool = True):
        # qkv: (b, num_tokens, dim)
        return super().forward(qkv, qkv, qkv, channels_first)

In [ ]:
test = TransformerEncoderBlock3D(dim=54, num_heads=6, mlp_ratio=2)
display(test)
o = test(torch.randn(2, 54, 4, 4, 4))
display(o.shape)


TransformerEncoderBlock3D(
  (attn): Attention3D(
    (W_q): Linear(in_features=54, out_features=54, bias=True)
    (W_k): Linear(in_features=54, out_features=54, bias=True)
    (W_v): Linear(in_features=54, out_features=54, bias=True)
    (proj): Linear(in_features=54, out_features=54, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm1): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
  (mlp): Attention3DMLP(
    (dense1): Linear(in_features=54, out_features=108, bias=True)
    (act): GELU(approximate='none')
    (dense2): Linear(in_features=108, out_features=54, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (layernorm2): LayerNorm((54,), eps=1e-06, elementwise_affine=True)
  (residual): Residual()
)

torch.Size([2, 54, 4, 4, 4])

In [ ]:
# | export


class TransformerDecoderBlock1D(nn.Module):
    def __init__(self, config: Attention1DWithMLPConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = Attention1DWithMLPConfig.model_validate(config | kwargs)

        dim = self.config.dim
        num_heads = self.config.num_heads
        mlp_ratio = self.config.mlp_ratio
        layer_norm_eps = self.config.layer_norm_eps
        attn_drop_prob = self.config.attn_drop_prob
        proj_drop_prob = self.config.proj_drop_prob
        mlp_drop_prob = self.config.mlp_drop_prob

        self.attn1 = Attention1D(
            dim=dim,
            num_heads=num_heads,
            attn_drop_prob=attn_drop_prob,
            proj_drop_prob=proj_drop_prob,
        )
        self.layernorm1 = nn.LayerNorm(dim, eps=layer_norm_eps)
        self.attn2 = Attention1D(
            dim=dim,
            num_heads=num_heads,
            attn_drop_prob=attn_drop_prob,
            proj_drop_prob=proj_drop_prob,
        )
        self.layernorm2 = nn.LayerNorm(dim, eps=layer_norm_eps)
        self.mlp = Attention1DMLP(dim=dim, mlp_ratio=mlp_ratio, mlp_drop_prob=mlp_drop_prob)
        self.layernorm3 = nn.LayerNorm(dim, eps=layer_norm_eps)

        self.residual = Residual()

        self.checkpointing_level3 = ActivationCheckpointing(3, checkpointing_level)

    def _forward(self, q: torch.Tensor, kv: torch.Tensor):
        # q: (b, num_tokens_in_q, dim)
        # kv: (b, num_tokens_in_kv, dim)

        res_connection1 = q
        # (b, num_tokens_in_q, dim)

        if self.config.norm_location == "pre":
            q = self.layernorm1(q)
            # (b, num_tokens_in_q, dim)
            kv = self.layernorm1(kv)
            # (b, num_tokens_in_kv, dim)

        hidden_states = self.attn1(q, q, q)
        # (b, num_tokens_in_q, dim)

        if self.config.norm_location == "post":
            hidden_states = self.layernorm1(hidden_states)
            # (b, num_tokens_in_q, dim)

        hidden_states = self.residual(res_connection1, hidden_states)
        res_connection2 = hidden_states
        # (b, num_tokens_in_q, dim)

        if self.config.norm_location == "pre":
            hidden_states = self.layernorm2(hidden_states)
            # (b, num_tokens_in_q, dim)

        hidden_states = self.attn2(hidden_states, kv, kv)
        # (b, num_tokens_in_q, dim)

        if self.config.norm_location == "post":
            hidden_states = self.layernorm2(hidden_states)
            # (b, num_tokens_in_q, dim)

        hidden_states = self.residual(res_connection2, hidden_states)
        res_connection3 = hidden_states
        # (b, num_tokens_in_q, dim)

        if self.config.norm_location == "pre":
            hidden_states = self.layernorm3(hidden_states)
            # (b, num_tokens_in_q, dim)

        hidden_states = self.mlp(hidden_states)
        # (b, num_tokens_in_q, dim)

        if self.config.norm_location == "post":
            hidden_states = self.layernorm3(hidden_states)
            # (b, num_tokens_in_q, dim)

        hidden_states = self.residual(res_connection3, hidden_states)
        # (b, num_tokens_in_q, dim)

        return hidden_states

    def forward(self, *args, **kwargs):
        return self.checkpointing_level3(self._forward, *args, **kwargs)

In [ ]:
test = TransformerDecoderBlock1D(dim=52, num_heads=4, mlp_ratio=2)
display(test)
display(test(torch.randn(2, 64, 52), torch.randn(2, 64, 52)).shape)


TransformerDecoderBlock1D(
  (attn1): Attention1D(
    (W_q): Linear(in_features=52, out_features=52, bias=True)
    (W_k): Linear(in_features=52, out_features=52, bias=True)
    (W_v): Linear(in_features=52, out_features=52, bias=True)
    (proj): Linear(in_features=52, out_features=52, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm1): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
  (attn2): Attention1D(
    (W_q): Linear(in_features=52, out_features=52, bias=True)
    (W_k): Linear(in_features=52, out_features=52, bias=True)
    (W_v): Linear(in_features=52, out_features=52, bias=True)
    (proj): Linear(in_features=52, out_features=52, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm2): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
  (mlp): Attention1DMLP(
    (dense1): Linear(in_features=52, out_features=104, bias=True)
    (act): GELU(approximate='none')
    (dense2): Linear(in_features=104, out_features=52, bias

torch.Size([2, 64, 52])

In [ ]:
# | export


class TransformerDecoderBlock3D(nn.Module):
    def __init__(self, config: Attention3DWithMLPConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = Attention3DWithMLPConfig.model_validate(config | kwargs)

        dim = self.config.dim
        num_heads = self.config.num_heads
        mlp_ratio = self.config.mlp_ratio
        layer_norm_eps = self.config.layer_norm_eps
        attn_drop_prob = self.config.attn_drop_prob
        proj_drop_prob = self.config.proj_drop_prob
        mlp_drop_prob = self.config.mlp_drop_prob

        self.attn1 = Attention3D(
            dim=dim,
            num_heads=num_heads,
            attn_drop_prob=attn_drop_prob,
            proj_drop_prob=proj_drop_prob,
        )
        self.layernorm1 = nn.LayerNorm(dim, eps=layer_norm_eps)
        self.attn2 = Attention3D(
            dim=dim,
            num_heads=num_heads,
            attn_drop_prob=attn_drop_prob,
            proj_drop_prob=proj_drop_prob,
        )
        self.layernorm2 = nn.LayerNorm(dim, eps=layer_norm_eps)
        self.mlp = Attention3DMLP(dim=dim, mlp_ratio=mlp_ratio, mlp_drop_prob=mlp_drop_prob)
        self.layernorm3 = nn.LayerNorm(dim, eps=layer_norm_eps)

        self.residual = Residual()

        self.checkpointing_level3 = ActivationCheckpointing(3, checkpointing_level)

    def _forward(self, q: torch.Tensor, kv: torch.Tensor, channels_first: bool = True):
        # Each is (b, [dim], tokens_z, tokens_y, tokens_x, [dim])

        if channels_first:
            q = rearrange(q, "b d z y x -> b z y x d").contiguous()
            kv = rearrange(kv, "b d z y x -> b z y x d").contiguous()
            # (b, tokens_z, tokens_y, tokens_x, dim)

        res_connection1 = q
        # (b, tokens_z, tokens_y, tokens_x, dim)

        if self.config.norm_location == "pre":
            q = self.layernorm1(q)
            kv = self.layernorm1(kv)
            # (b, tokens_z, tokens_y, tokens_x, dim)

        hidden_states: torch.Tensor = self.attn1(q, q, q, channels_first=False)
        # (b, tokens_z, tokens_y, tokens_x, dim)

        if self.config.norm_location == "post":
            hidden_states = self.layernorm1(hidden_states)
            # (b, tokens_z, tokens_y, tokens_x, dim)

        hidden_states = self.residual(res_connection1, hidden_states)
        res_connection2 = hidden_states
        # (b, tokens_z, tokens_y, tokens_x, dim)

        if self.config.norm_location == "pre":
            hidden_states = self.layernorm2(hidden_states)
            # (b, tokens_z, tokens_y, tokens_x, dim)

        hidden_states = self.attn2(hidden_states, kv, kv, channels_first=False)
        # (b, tokens_z, tokens_y, tokens_x, dim)

        if self.config.norm_location == "post":
            hidden_states = self.layernorm2(hidden_states)
            # (b, tokens_z, tokens_y, tokens_x, dim)

        hidden_states = self.residual(res_connection2, hidden_states)
        res_connection3 = hidden_states
        # (b, tokens_z, tokens_y, tokens_x, dim)

        if self.config.norm_location == "pre":
            hidden_states = self.layernorm3(hidden_states)
            # (b, tokens_z, tokens_y, tokens_x, dim)

        hidden_states = self.mlp(hidden_states, channels_first=False)
        # (b, tokens_z, tokens_y, tokens_x, dim)

        if self.config.norm_location == "post":
            hidden_states = self.layernorm3(hidden_states)
            # (b, tokens_z, tokens_y, tokens_x, dim)

        hidden_states = self.residual(res_connection3, hidden_states)
        # (b, tokens_z, tokens_y, tokens_x, dim)

        if channels_first:
            hidden_states = rearrange(hidden_states, "b z y x d -> b d z y x").contiguous()
            # (b, dim, tokens_z, tokens_y, tokens_x)

        return hidden_states

    def forward(self, *args, **kwargs):
        return self.checkpointing_level3(self._forward, *args, **kwargs)

In [ ]:
test = TransformerDecoderBlock3D(dim=52, num_heads=4, mlp_ratio=2)
display(test)
display(test(torch.randn(2, 52, 4, 4, 4), torch.randn(2, 52, 4, 4, 4)).shape)


TransformerDecoderBlock3D(
  (attn1): Attention3D(
    (W_q): Linear(in_features=52, out_features=52, bias=True)
    (W_k): Linear(in_features=52, out_features=52, bias=True)
    (W_v): Linear(in_features=52, out_features=52, bias=True)
    (proj): Linear(in_features=52, out_features=52, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm1): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
  (attn2): Attention3D(
    (W_q): Linear(in_features=52, out_features=52, bias=True)
    (W_k): Linear(in_features=52, out_features=52, bias=True)
    (W_v): Linear(in_features=52, out_features=52, bias=True)
    (proj): Linear(in_features=52, out_features=52, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (layernorm2): LayerNorm((52,), eps=1e-06, elementwise_affine=True)
  (mlp): Attention3DMLP(
    (dense1): Linear(in_features=52, out_features=104, bias=True)
    (act): GELU(approximate='none')
    (dense2): Linear(in_features=104, out_features=52, bias

torch.Size([2, 52, 4, 4, 4])

# nbdev

In [ ]:
!nbdev_export